In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("selfjoin").getOrCreate()


25/02/08 15:53:00 WARN Utils: Your hostname, jeevas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.101 instead (on interface en0)
25/02/08 15:53:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/08 15:53:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

In [4]:
empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [5]:
dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]

In [6]:
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [14]:
# Self Join DataFrames
from pyspark.sql.functions import col

empDF.alias("emp1").join(empDF.alias("emp2"), \
    col("emp1.superior_emp_id") == col("emp2.emp_id"),"inner") \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name"), \
      col("emp1.year_joined"), \
      col("emp1.emp_dept_id"), \
      col("emp1.gender"), \
      col("emp1.salary")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|superior_emp_name|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------------+-----------+-----------+------+------+
|2     |Rose    |1              |Smith            |2010       |20         |M     |4000  |
|3     |Williams|1              |Smith            |2010       |10         |M     |1000  |
|4     |Jones   |2              |Rose             |2005       |10         |F     |2000  |
|5     |Brown   |2              |Rose             |2010       |40         |      |-1    |
|6     |Brown   |2              |Rose             |2010       |50         |      |-1    |
+------+--------+---------------+-----------------+-----------+-----------+------+------+



In [15]:
# Self Join DataFrames
from pyspark.sql.functions import col

empDF.alias("emp1").join(empDF.alias("emp2"), \
    col("emp1.superior_emp_id") == col("emp2.emp_id"),"inner") \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name"), \
      col("emp1.year_joined"), \
      col("emp1.emp_dept_id"), \
      col("emp2.gender"), \
      col("emp1.salary")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|superior_emp_name|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------------+-----------+-----------+------+------+
|2     |Rose    |1              |Smith            |2010       |20         |M     |4000  |
|3     |Williams|1              |Smith            |2010       |10         |M     |1000  |
|4     |Jones   |2              |Rose             |2005       |10         |M     |2000  |
|5     |Brown   |2              |Rose             |2010       |40         |M     |-1    |
|6     |Brown   |2              |Rose             |2010       |50         |M     |-1    |
+------+--------+---------------+-----------------+-----------+-----------+------+------+



In [16]:
# Self Join DataFrames
from pyspark.sql.functions import col

empDF.alias("emp1").join(empDF.alias("emp2"), \
    col("emp1.superior_emp_id") == col("emp2.emp_id"),"left") \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name"), \
      col("emp1.year_joined"), \
      col("emp1.emp_dept_id"), \
      col("emp2.gender"), \
      col("emp1.salary")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|superior_emp_name|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------------+-----------+-----------+------+------+
|1     |Smith   |NULL           |NULL             |2018       |10         |NULL  |3000  |
|2     |Rose    |1              |Smith            |2010       |20         |M     |4000  |
|3     |Williams|1              |Smith            |2010       |10         |M     |1000  |
|4     |Jones   |2              |Rose             |2005       |10         |M     |2000  |
|5     |Brown   |2              |Rose             |2010       |40         |M     |-1    |
|6     |Brown   |2              |Rose             |2010       |50         |M     |-1    |
+------+--------+---------------+-----------------+-----------+-----------+------+------+



In [9]:
# Self Join DataFrames
from pyspark.sql.functions import col

empDF.alias("emp1").join(empDF.alias("emp2"), \
    col("emp1.superior_emp_id") == col("emp2.emp_id"),"left") \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+
|emp_id|name    |superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|1     |Smith   |NULL           |NULL             |
|2     |Rose    |1              |Smith            |
|3     |Williams|1              |Smith            |
|4     |Jones   |2              |Rose             |
|5     |Brown   |2              |Rose             |
|6     |Brown   |2              |Rose             |
+------+--------+---------------+-----------------+



In [8]:
# Self Join using SQL
empDF.createOrReplaceTempView("EMP")

joinDF2 = spark.sql("SELECT e1.emp_id, e1.name, e2.name as superior_name FROM EMP e1 INNER JOIN EMP e2 ON e1.emp_id = e2.superior_emp_id ") \
  .show(truncate=False)

+------+-----+-------------+
|emp_id|name |superior_name|
+------+-----+-------------+
|1     |Smith|Rose         |
|1     |Smith|Williams     |
|2     |Rose |Jones        |
|2     |Rose |Brown        |
|2     |Rose |Brown        |
+------+-----+-------------+

